### *Week 3 Assignment: 3*
## Segmenting and Clustering Neighborhoods in Toronto:

### Instaling & importing packages:

In [1]:
MAX_RECORDS = 1000
!pip install geopy
!pip install folium

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline 

mpl.style.use('ggplot') # optional: for ggplot-like style
# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Libraries imported.")

     |████████████████████████████████| 92kB 10.2MB/s eta 0:00:01
Libraries imported.


### Importing result data from Task2:

In [2]:
toronto_df = pd.read_csv('part_2.csv')

toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


### Determining the number of unique Boroughs and Neighborhoods in Totronto:

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Visualizing Boroughs & Neighborhoods in Toronto using Folium Map:

In [4]:
toronto_map = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Preparing data will be used for Clustering:

In [5]:
toronto_df1 = toronto_df.drop(['PostalCode', 'Borough', 'Neighborhood'], axis=1)
toronto_df1.head()

,Latitude,Longitude
0,43.752440,-79.329271
1,43.730421,-79.313320
2,43.655120,-79.362640
3,43.723125,-79.451589
4,43.661102,-79.391035


### Clustering using the K-Mean method with 4 Clusters:

In [6]:
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12, random_state=0).fit(toronto_df1)
Clusters = k_means.labels_
colors = ['red', 'green', 'blue', 'yellow']
toronto_df['Cluster'] = Clusters

In [7]:


for latitude, longitude, borough, cluster in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

In [8]:
toronto_df.to_csv('3_week_part_3.csv', index=False)